In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
model_name = 'K80'
file_name = 'K80'

In [8]:
DATA_DIR = os.path.abspath("../../../prediction_model_tf2/practicum_data/raw_data")
SAVE_DIR = os.path.abspath("../../../prediction_model_tf2/practicum_data/")

## Dense Layer

In [4]:
dfDense = pd.read_pickle(os.path.join(DATA_DIR,'%s/8/benchmark_dense__20180907.pkl' %file_name))

for i in range(9,10):
    dfDense = pd.concat([dfDense,pd.read_pickle(os.path.join(DATA_DIR,'%s/%d/benchmark_dense__20180907.pkl' %(file_name, i)))])

In [5]:
ops = (dfDense['batchsize']
       * dfDense['dim_input'] 
       * dfDense['dim_output'])

memory_weights = dfDense['dim_input'] * dfDense['dim_output']
memory_in = dfDense['batchsize'] * dfDense['dim_input']
memory_out = dfDense['batchsize'] * dfDense['dim_output']


dfDense['optimizer'] = dfDense['optimizer'].replace({0:'opt_None',
                                                     1:'opt_SGD',
                                                     2:'opt_Adadelta',
                                                     3:'opt_Adagrad',
                                                     4:'opt_Momentum',
                                                     5:'opt_Adam',
                                                     6:'opt_RMSProp'})

dfDense['activation_fct'] = dfDense['activation_fct'].replace({0:'act_None',
                                                               1:'act_relu',
                                                               2:'act_tanh',
                                                               3:'act_sigmoid'})

dfDense['ops'] = ops
dfDense['memory_weights'] = memory_weights
dfDense['memory_in'] = memory_in
dfDense['memory_out'] = memory_out

In [6]:
one_hot_optimizer = pd.get_dummies(dfDense['optimizer'])

dfDense = dfDense.drop(labels='optimizer',axis=1)
dfDense = pd.concat([dfDense,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfDense['activation_fct'])

dfDense = dfDense.drop(labels='activation_fct',axis=1)
dfDense = pd.concat([dfDense,one_hot_activation],axis=1)

In [7]:
dfDense.describe()

,batchsize,dim_input,dim_output,gpu,precision,timeUsed_max,timeUsed_median,timeUsed_min,timeUsed_std,ops,...,opt_Adagrad,opt_Adam,opt_Momentum,opt_None,opt_RMSProp,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
count,25000.000000,25000.000000,25000.000000,25000.0,25000.0,25000.000000,25000.000000,25000.000000,25000.000000,2.500000e+04,...,25000.000000,25000.000000,25000.000000,25000.00000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000
mean,32.675880,2054.253560,2061.138840,0.0,32.0,1.712056,1.521588,1.479327,0.088182,1.385280e+08,...,0.084160,0.081120,0.083040,0.50324,0.085320,0.082360,0.248520,0.250600,0.251640,0.249240
std,18.454247,1185.428045,1186.325093,0.0,0.0,1.657588,1.565794,1.529975,0.099323,1.597647e+08,...,0.277633,0.273025,0.275948,0.50000,0.279363,0.274918,0.432163,0.433367,0.433964,0.432582
min,1.000000,1.000000,1.000000,0.0,32.0,0.212402,0.211139,0.199342,0.000813,6.540000e+02,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,17.000000,1019.000000,1041.000000,0.0,32.0,0.587395,0.484552,0.467534,0.022344,2.337678e+07,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,33.000000,2065.000000,2070.000000,0.0,32.0,1.049328,0.878060,0.852039,0.053896,7.763155e+07,...,0.000000,0.000000,0.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,49.000000,3079.000000,3087.000000,0.0,32.0,2.205945,1.946273,1.895158,0.119896,1.949170e+08,...,0.000000,0.000000,0.000000,1.00000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
max,64.000000,4095.000000,4095.000000,0.0,32.0,10.785599,10.292978,10.209756,1.515937,9.754019e+08,...,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
dfDense.to_pickle(os.path.join(SAVE_DIR,'Data_dense_%s.pkl'%model_name))

## Convolutional Layers

In [9]:
file_list = os.listdir(os.path.join(DATA_DIR, file_name))
dfConv = pd.read_pickle(os.path.join(DATA_DIR,'%s/%s' %(file_name, file_list[0])))

header = dfConv.columns

for i in range(1, len(file_list)):
    dfConv = pd.concat([dfConv,pd.read_pickle(os.path.join(DATA_DIR,'%s/%s' %(file_name, file_list[i])))])
dfConv

,batchsize,matsize,kernelsize,channels_in,channels_out,strides,padding,precision,activation_fct,use_bias,optimizer,gpu,timeUsed_median,timeUsed_min,timeUsed_max,timeUsed_std
0,3,445,6,12,5,2,1,32,3,True,2,0,20.644941,20.547748,21.568322,0.381334
1,8,17,1,422,149,3,1,32,2,False,6,0,1.170106,1.116323,1.219935,0.035430
2,53,37,1,201,93,4,1,32,3,False,1,0,5.051761,5.013375,5.062790,0.017356
3,48,465,5,15,11,3,1,32,2,True,4,0,203.171034,201.529980,203.245697,0.667110
4,11,386,1,13,18,3,1,32,3,False,0,0,3.035989,3.024130,3.110862,0.031102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,25,62,1,145,146,1,0,32,2,False,0,0,7.211471,7.180233,7.263651,0.029507
2496,32,453,5,17,14,1,1,32,1,True,5,0,431.072474,430.990601,431.167169,0.055987
2497,17,21,6,267,256,2,0,32,1,False,2,0,15.118537,15.070500,15.181332,0.041994
2498,28,344,1,2,14,2,1,32,2,True,0,0,5.874748,5.851150,5.890617,0.014478


In [10]:
padding_reduction = ((dfConv['padding']==0)
                     *(dfConv['kernelsize']-1))

elements_output = ((dfConv['matsize'] - padding_reduction)
                   / dfConv['strides'])**2

ops = (dfConv['batchsize']
       * elements_output
       * dfConv['kernelsize']**2
       * dfConv['channels_in']
       * dfConv['channels_out'])

memory_weights = (dfConv['kernelsize']**2
                  * dfConv['channels_in']
                  * dfConv['channels_out']
                  + dfConv['use_bias'] * dfConv['channels_out'])

memory_in = (dfConv['batchsize']
             * dfConv['matsize']**2
             * dfConv['channels_in'])

memory_out = (dfConv['batchsize']
              * elements_output
              * dfConv['channels_out'])


dfConv['elements_matrix'] = dfConv['matsize']**2
dfConv['elements_kernel'] = dfConv['kernelsize']**2

    
dfConv['ops'] = ops
dfConv['memory_weights'] = memory_weights
dfConv['memory_in'] = memory_in
dfConv['memory_out'] = memory_out


dfConv['optimizer'] = dfConv['optimizer'].replace({0:'opt_None',
                                                   1:'opt_SGD',
                                                   2:'opt_Adadelta',
                                                   3:'opt_Adagrad',
                                                   4:'opt_Momentum',
                                                   5:'opt_Adam',
                                                   6:'opt_RMSProp'})

dfConv['activation_fct'] = dfConv['activation_fct'].replace({0:'act_None',
                                                             1:'act_relu',
                                                             2:'act_tanh',
                                                             3:'act_sigmoid'})

dfConv['use_bias'] = np.uint8(dfConv['use_bias'])


dfConv.dropna(inplace=True)

In [11]:
one_hot_optimizer = pd.get_dummies(dfConv['optimizer'])

dfConv = dfConv.drop(labels='optimizer',axis=1)
dfConv = pd.concat([dfConv,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfConv['activation_fct'])

dfConv = dfConv.drop(labels='activation_fct',axis=1)
dfConv = pd.concat([dfConv,one_hot_activation],axis=1)

In [12]:
dfConv.describe()

,batchsize,matsize,kernelsize,channels_in,channels_out,strides,padding,precision,use_bias,gpu,...,opt_Adagrad,opt_Adam,opt_Momentum,opt_None,opt_RMSProp,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
count,2495.000000,2495.000000,2495.000000,2495.000000,2495.000000,2495.000000,2495.000000,2495.0,2495.000000,2495.0,...,2495.000000,2495.000000,2495.000000,2495.000000,2495.000000,2495.000000,2495.000000,2495.000000,2495.000000,2495.000000
mean,32.690982,259.349900,4.006012,72.661723,69.202004,2.473747,0.499399,32.0,0.492184,0.0,...,0.084569,0.080160,0.086172,0.486573,0.087776,0.089379,0.254509,0.246894,0.247695,0.250902
std,18.567922,149.757207,2.013080,404.084145,318.164860,1.129191,0.500100,0.0,0.500039,0.0,...,0.278295,0.271596,0.280675,0.499920,0.283025,0.285347,0.435672,0.431291,0.431760,0.433619
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,32.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,16.500000,130.000000,2.000000,10.000000,9.000000,1.000000,0.000000,32.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,33.000000,258.000000,4.000000,18.000000,18.000000,2.000000,0.000000,32.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,49.000000,393.000000,6.000000,38.000000,39.000000,4.000000,1.000000,32.0,1.000000,0.0,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,64.000000,512.000000,7.000000,9758.000000,7072.000000,4.000000,1.000000,32.0,1.000000,0.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
dfConv.to_pickle(os.path.join(SAVE_DIR,'Data_convolution_%s.pkl'%model_name))